# ALY6040 Final Project

## Exploratory Data Analysis

In [1]:
import pandas as pd
import numpy as np
from plotly import express as px
from plotly import figure_factory as ff
from plotly import graph_objects as go

In [2]:
# Load the dataset
df = pd.read_csv("final_project_dataset.csv", low_memory=False)

### 1. Initial Exploration

In [3]:
print("Dataset Info:")
print(df.info())

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182242 entries, 0 to 182241
Data columns (total 65 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   PID                  182242 non-null  int64  
 1   CM_ID                93291 non-null   float64
 2   GIS_ID               182242 non-null  int64  
 3   ST_NUM               172879 non-null  float64
 4   ST_NAME              182242 non-null  object 
 5   UNIT_NUM             82613 non-null   object 
 6   CITY                 182239 non-null  object 
 7   ZIP_CODE             182239 non-null  float64
 8   BLDG_SEQ             182242 non-null  int64  
 9   NUM_BLDGS            182242 non-null  int64  
 10  LUC                  182242 non-null  int64  
 11  LU                   182242 non-null  object 
 12  LU_DESC              182242 non-null  object 
 13  BLDG_TYPE            179626 non-null  object 
 14  OWN_OCC              182242 non-null  object 
 15  OWN

In [4]:
print("Summary Statistics:")
print(df.describe())

Summary Statistics:
                PID         CM_ID        GIS_ID         ST_NUM       ZIP_CODE  \
count  1.822420e+05  9.329100e+04  1.822420e+05  172879.000000  182239.000000   
mean   1.140093e+09  9.175609e+08  1.140094e+09     226.431695    2129.867937   
std    7.091114e+08  6.896463e+08  7.091123e+08     475.704566      30.721915   
min    1.000010e+08  1.000180e+08  1.000010e+08       0.000000    2026.000000   
25%    5.015880e+08  3.069060e+08  5.015882e+08      24.000000    2119.000000   
50%    1.102618e+09  6.026424e+08  1.102618e+09      68.000000    2127.000000   
75%    1.810508e+09  1.602331e+09  1.810508e+09     212.000000    2131.000000   
max    2.205670e+09  2.205665e+09  2.205670e+09    5341.000000    2467.000000   

       BLDG_SEQ      NUM_BLDGS            LUC      RES_FLOOR      CD_FLOOR  \
count  182242.0  182242.000000  182242.000000  148450.000000  71972.000000   
mean        1.0       1.000077     202.581644       1.880354      3.520313   
std         0.0 

In [5]:
print("Missing Values")
print((((df.isna().sum()) * 100 ) / df.shape[0]).sort_values(ascending=False).head(20))

Missing Values
COM_UNITS          94.091373
RES_UNITS          94.091373
RC_UNITS           94.091373
KITCHEN_STYLE3     92.457831
STRUCTURE_CLASS    90.448963
KITCHEN_STYLE2     82.853568
MAIL_ADDRESSEE     81.117415
BTHRM_STYLE3       79.970589
BDRM_COND          60.633663
CORNER_UNIT        60.508006
CD_FLOOR           60.507457
ORIENTATION        60.506360
HEAT_SYSTEM        60.366436
UNIT_NUM           54.668518
BTHRM_STYLE2       53.268182
YR_REMODEL         52.416018
CM_ID              48.809276
KITCHEN_TYPE       27.191866
KITCHEN_STYLE1     27.188573
BTHRM_STYLE1       27.188025
dtype: float64


### 2. Handle Missing Values

In [6]:
# Clean column names
df.columns = df.columns.str.strip().str.lower()

In [7]:
# Convert numeric columns
numeric_columns = ["land_sf", "land_value", "bldg_value", "total_value", "sfyi_value", "yr_built", "yr_remodel"]
for col in numeric_columns:
    df[col] = df[col].astype(str).str.replace(r"[^0-9.]", "", regex=True)
    df[col] = pd.to_numeric(df[col], errors="coerce")

In [8]:
# Drop high-missing columns
drop_cols = ["res_units", "com_units", "rc_units", "structure_class"]
df.drop(columns=drop_cols, inplace=True)

In [9]:
# Property classification
df['property_type'] = ''
residential = ["R1", "R2", "R3", "R4", "A", "CD", "CM", "RL"]
commercial = ["CC", "C", "CL"]
industrial = ["I"]
mixed_use = ["RC"]

df.loc[df["lu"].isin(residential), "property_type"] = "Residential"
df.loc[df["lu"].isin(commercial), "property_type"] = "Commercial"
df.loc[df["lu"].isin(industrial), "property_type"] = "Industrial"
df.loc[df["lu"].isin(mixed_use), "property_type"] = "Mixed-Use"
df['property_type'].fillna("Other", inplace=True)

<ipython-input-9-e54b79a643c9>:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['property_type'].fillna("Other", inplace=True)


In [10]:
# Impute missing values based on property type
res_cols = ["bed_rms", "full_bth", "hlf_bth", "kitchens", "tt_rms", "living_area", "gross_area", "yr_built"]
for col in res_cols:
    df.loc[df["property_type"] == "Residential", col] = df.loc[df["property_type"] == "Residential", col].fillna(df[col].median())

df.loc[df["property_type"] != "Residential", res_cols] = np.nan

In [11]:
# Fill categorical missing values
categorical_cols = ["bldg_type", "heat_type", "ac_type", "prop_view"]
for col in categorical_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)

<ipython-input-11-022ee220dad6>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)


In [12]:
# Feature Engineering
df["value_per_sqft"] = df["total_value"] / df["land_sf"]

In [13]:
# Drop rows with missing key values before plotting
df_clean = df.dropna(subset=["yr_built", "total_value", "bldg_value", "land_value"])

### Hypothesis 1: Building Value vs. Land Value vs. Total Value (Scatter Plot)

In [14]:
# fig1 = go.Figure()
# fig1.add_trace(go.Scatter(x=df_clean["bldg_value"], y=df_clean["total_value"], mode="markers",
#                           name="Building Value", marker=dict(color="blue", opacity=0.5)))
# fig1.add_trace(go.Scatter(x=df_clean["land_value"], y=df_clean["total_value"], mode="markers",
#                           name="Land Value", marker=dict(color="green", opacity=0.5)))
# fig1.update_layout(title="Building vs. Land Contribution to Total Property Value",
#                    xaxis_title="Value ($)", yaxis_title="Total Property Value ($)")
# fig1.show()

### Hypothesis 2: Property Age & Remodeling Impact on Value

In [15]:
# fig2 = px.box(df_clean, x="property_type", y="total_value", color=df_clean["yr_built"].astype(str),
#               title="Property Value by Age and Remodel Status",
#               labels={"property_type": "Property Type", "total_value": "Total Assessed Value ($)", "yr_built": "Year Built"})

# fig2.show()

### Hypothesis 3: Residential Properties have a higher Value per SQFT as compared to Industrial or Commercial

In [16]:
# fig3 = go.Figure()
# fig3.add_trace(go.Bar(x=df["property_type"].unique(), y=df.groupby("property_type")["total_value"].mean(),
#                       name="Total Value", marker_color="blue"))
# fig3.add_trace(go.Bar(x=df["property_type"].unique(), y=df.groupby("property_type")["value_per_sqft"].mean(),
#                       name="Value per Sq Ft", marker_color="orange"))
# fig3.update_layout(title="Commercial vs. Residential: Total Value vs. Value per Sq Ft",
#                    xaxis_title="Property Type", yaxis_title="Value ($)", barmode="group")

# fig3.show()

In [17]:
print("Missing Values")
print((((df.isna().sum()) * 100 ) / df.shape[0]).sort_values(ascending=False).head(30))

Missing Values
kitchen_style3    92.457831
kitchen_style2    82.853568
mail_addressee    81.117415
bthrm_style3      79.970589
bdrm_cond         60.633663
corner_unit       60.508006
cd_floor          60.507457
orientation       60.506360
heat_system       60.366436
unit_num          54.668518
bthrm_style2      53.268182
yr_remodel        52.416018
cm_id             48.809276
kitchen_type      27.191866
kitchen_style1    27.188573
bthrm_style1      27.188025
fireplaces        27.180343
int_wall          26.749597
int_cond          26.747951
num_parking       26.680458
roof_structure    19.877416
roof_cover        19.874123
ext_cond          19.840651
res_floor         18.542378
tt_rms            18.274053
bed_rms           18.274053
full_bth          18.274053
kitchens          18.274053
yr_built          18.274053
gross_area        18.274053
dtype: float64


In [18]:
df['kitchen_style3'] =df['kitchen_style3'].fillna('No Kitchen 3')
df['kitchen_style2'] =df['kitchen_style2'].fillna('No Kitchen 2')
df['bthrm_style3'] =df['bthrm_style3'].fillna('No Bathroom 3')
df['bthrm_style2'] =df['bthrm_style2'].fillna('No Bathroom 2')

In [19]:
df = df.drop(columns=['mail_addressee', 'unit_num', 'cm_id', 'gis_id', 'corner_unit', 'cd_floor', 'orientation', 'mail_state', 'mail_zip_code', 'mail_city', 'mail_street_address'])

In [20]:
df['bdrm_cond'] = df['bdrm_cond'].fillna('Not Specified')
df['int_cond'] = df['int_cond'].fillna('Not Specified')
df['ext_cond'] = df['ext_cond'].fillna('Not Specified')
df['overall_cond'] = df['overall_cond'].fillna('Not Specified')
df['heat_system'] =df['heat_system'].fillna('Not Specified')
df['int_wall'] =df['int_wall'].fillna('Not Specified')

In [21]:
df = df.dropna(subset=['land_sf', 'gross_area', 'living_area', 'num_parking', 'st_num', 'roof_structure', 'roof_cover', 'ext_fnished', 'res_floor'])

In [22]:
df['yr_remodel'] = df['yr_remodel'].fillna(-1)

In [23]:
df['kitchen_type'] = df['kitchen_type'].fillna('Not Specified')
df['kitchen_style1'] = df['kitchen_type'].fillna('No Kitchen 1')
df['bthrm_style1'] = df['bthrm_style1'].fillna('No Bathroom 1')

In [24]:
df['fireplaces'] = df['fireplaces'].fillna(0.0)


### 3. Model Building

In [25]:
df['gross_tax'] = df['gross_tax'].astype(str).str.replace('$', '', regex=False)
df['gross_tax'] = df['gross_tax'].str.replace(',', '', regex=False)
df['gross_tax'] = df['gross_tax'].str.replace(' ', '', regex=False)
df['gross_tax'] = pd.to_numeric(df['gross_tax'])

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
cat_features = [df.columns.get_loc(col) for col in categorical_cols]
X = df.drop('total_value', axis=1)
y = df['total_value']

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
df = df.drop(columns=['owner', 'st_name', 'st_num', 'lu_desc', 'pid', 'bldg_seq'])

In [37]:
df_dummies = pd.get_dummies(df)

In [38]:
X = df_dummies.drop('total_value', axis=1)
y = df_dummies['total_value']

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [46]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor.fit(X_train, y_train)

y_pred = rf_regressor.predict(X_test)

In [47]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Squared Error: 2139990414.2754748
R-squared: 0.997012767061803


In [54]:
feature_importances = rf_regressor.feature_importances_ * 100
feature_importance_df = pd.DataFrame({
       'Feature': X_train.columns,
       'Importance': feature_importances
   })
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

In [59]:
feature_importance_df
fig = px.bar(
    data_frame=feature_importance_df.head(10),
    x = 'Feature',
    y = 'Importance'
)
fig.show()

In [60]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

y_pred_lr = lr_model.predict(X_test)

In [61]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Squared Error: 2139990414.2754748
R-squared: 0.997012767061803


In [65]:
lr_model.coef_

array([ 1.12129261e-05, -3.23306133e-03, -1.57802498e-03, -1.21200646e-03,
        3.01186921e-07,  9.68520310e-07,  6.06319642e-07,  3.17305043e-06,
        3.17876039e-06, -2.70006240e-12,  9.17428276e+01, -7.30030670e-06,
        7.53730262e-07,  1.53743791e-04, -1.10217344e-03, -2.15962108e-03,
       -8.30087052e-05, -3.73403139e-05,  2.33436550e-04,  1.46985812e-04,
        9.33886735e-06,  4.57168330e-03,  2.15871536e-03,  2.59261696e-03,
       -3.10384879e-03,  6.31983072e-03, -8.79780885e-03,  2.04352201e-03,
       -2.16667392e-03,  1.53771494e-03, -1.27556543e-03,  3.94190954e-03,
       -1.99848442e-03, -1.16573418e-15, -5.45535973e-04, -4.66108594e-03,
       -4.19767920e-03,  4.15140940e-03, -5.70719708e-04,  2.50401868e-02,
       -7.11438073e-04, -1.64129432e-02, -6.08134643e-03, -5.44219661e-03,
        3.60773743e-03, -2.51637145e-02, -7.04515627e-03,  1.24307204e-01,
        4.64231857e-03,  6.10511771e-03,  5.72503800e-03, -7.67331403e-02,
        4.41567019e-03,  